In [1]:
# Run prediction and genertae pixelwise annotation for every pixels in the image using fully coonvolutional neural net
# Output saved as label images, and label image overlay on the original image
# 1) Make sure you you have trained model in logs_dir (See Train.py for creating trained model)
# 2) Set the Image_Dir to the folder where the input image for prediction are located
# 3) Set number of classes number in NUM_CLASSES
# 4) Set Pred_Dir the folder where you want the output annotated images to be save
# 5) Run script
#--------------------------------------------------------------------------------------------------------------------
import tensorflow as tf
import numpy as np
import scipy.misc as misc
import sys
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops

In [2]:

import BuildNetVgg16
import TensorflowUtils
import os
import Data_Reader
import OverrlayLabelOnImage as Overlay
import CheckVGG16Model
logs_dir= "logs/"# "path to logs directory where trained model and information will be stored"
Image_Dir="Data_Zoo/test/"# Test image folder
w=0.6# weight of overlay on image
Pred_Dir="Output_Prediction/" # Library where the output prediction will be written
model_path="Model_Zoo/vgg16.npy"# "Path to pretrained vgg16 model for encoder"
NameEnd="" # Add this string to the ending of the file name optional
NUM_CLASSES = 2 # Number of classes
#-------------------------------------------------------------------------------------------------------------------------
CheckVGG16Model.CheckVGG16(model_path)# Check if pretrained vgg16 model avialable and if not try to download it

In [3]:
import  imageio
Images = imageio.imread('Data_Zoo/test/acl2.jpg')
Images.shape
Images = Images[:,:,0:3]
Images.shape
import matplotlib.pyplot as plt
plt.imshow(Images)
plt.show()
Images = np.reshape(Images,[1,600,900,3])

In [4]:
@tf.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.where(0. < grad, gen_nn_ops._relu_grad(grad, op.outputs[0]), tf.zeros_like(grad))
#修改为grad和op都大于零

In [5]:
g = tf.get_default_graph()
with g.gradient_override_map({"Relu": "GuidedRelu"}):
    #load the net
    # .........................Placeholders for input image and labels........................................................................
    keep_prob = tf.placeholder(tf.float32, name="keep_probabilty")  # Dropout probability
    image = tf.placeholder(tf.float32, shape=[None, None, None, 3], name="input_image")  # Input image batch first dimension image number second dimension width third dimension height 4 dimension RGB
    # -------------------------Build Net----------------------------------------------------------------------------------------------
    Net = BuildNetVgg16.BUILD_NET_VGG16(vgg16_npy_path=model_path)  # Create class instance for the net
    Net.build(image, NUM_CLASSES, keep_prob)    
    
    #get input
    inpt = Net.bgr
    #get output
    outp = Net.Prob
    background, object = tf.split(outp,num_or_size_splits=2,axis=3)
    
    guided_back = tf.gradients(background,inpt)#决定可视化背景还是物体
    print(len(guided_back))
    guided_back_map = guided_back[0]
    sess1 = tf.Session()
    sess1.run(tf.global_variables_initializer())
    guided_back_map = sess1.run(guided_back_map, feed_dict={image: Images, keep_prob: 1.0})
        
print(guided_back_map.shape)

npy file loaded
build model started


Instructions for updating:
Use the `axis` argument instead


FCN model built


1


(1, 600, 900, 3)


In [6]:
guided_back_map = np.reshape(guided_back_map,[600,900,3])

# convert BGR to RGB
guided_back_map_rgb = np.zeros([600,900,3])
guided_back_map_rgb[:,:,0] = guided_back_map[:,:,2]
guided_back_map_rgb[:,:,1] = guided_back_map[:,:,1]
guided_back_map_rgb[:,:,2] = guided_back_map[:,:,0]
guided_back_map_abs = np.abs(guided_back_map_rgb).max(axis=-1)
plt.figure()
plt.imshow(guided_back_map_abs)
plt.show()
pos_guided_back = (np.maximum(0, guided_back_map_rgb) / guided_back_map_rgb.max())
plt.figure()
plt.imshow(pos_guided_back)
plt.show()
